# Cross referencing Google location history with MOH COVID-19 exposure locations
## Important notice!
> This code was **not** created with accuracy in mind, this is a POC and I really wanted to try my hand with pandas and used COVID-19 as an excuse. Please **do not** rely on the results.
I know for a fact that some locations that are marked as "clear" (location history indicated not in proximity of location) also have additional date and time information in the comment field.

## Notes
- This code calculates location history distance from exposure locations without any distance consideration.
- The code is not just **not optimized**, it bad, slow and unreadable.
- First time using pandas, didn't read the documentation. Also, not a programmer.
- It does work...

## Known bugs
- MOH timestamp fields are untrustworthy, therefore a combination of the timestamp and a stayTimes string  is used.
- If a stayTimes string is ends before it begins (such as "09:00 - 02:00") no corrections are made to the date and location is ignored.
- If the stayTimes string is "לא ידוע" ("Unknown"), there is not attempt done to select the full time range.
- MOH like to add additional exposure times in the comment field, I ignore them.

## Testing
With the supplied myData.json and govData.json, there should be 1169 cases of unknown locations marked orange on the map. Out of the 132 known locations, 130 should be ok (green) and 2 should be too close to an exposure location and marked with a red marker on the map (one in Ra'anana
 and another at the Ruppin academic centre).

## Logic behind the code
    None.

In [1]:
# imports
import numpy as np
import pandas as pd
import requests
import datetime as dt
import geopy.distance
import json
import os
from ipywidgets import IntSlider
from ipywidgets.embed import embed_minimal_html
from ipyleaflet import Map, FullScreenControl, Marker, basemaps, AwesomeIcon
import itertools


%matplotlib notebook

# Make page wider, better view of tabular data
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

### Asset acquisition
This will download the latest MOH exposure locations file. Also, make sure the "Location History.json" file from google
is located in the same directory under the name "myData.json". If I did not forget, there should be a sample "myData.json" file in the repository to run this notebook without actual data.

In [2]:
# COVID-19 exposure locations from MOH
GOV_CASE_LOC_URL = "https://gisweb.azureedge.net/Points.json"

def downloadFile(url, path):
    """
    Download a file and save it.
    """
    print(f"Requesting {url}")
    res = requests.get(url, stream=True)
    length = res.headers.get("content-length")
    print(f"Downloading")
    with open(path, 'wb') as f:
        if length is None:
            print("No content length.")
            f.write(res.content)
        else:
            length = int(length)
            dl_length = 0
            print(f"Total size: {length}")
            for data in res.iter_content(chunk_size=4096):
                dl_length += len(data)
                f.write(data)
            print(f"Downloaded: {dl_length}")

# Uncomment to download newest MOH dataset
#downloadFile(GOV_CASE_LOC_URL, 'govData.json')
if not os.path.isfile('myData.json'):
    # Copy the "Location History.json" file from Google takeout to here, rename it "myData.json"
    raise Exception("Missing location history file (myData.json). ")

### Google data normalization 
Load the google location history and put it into a known DataFrame with a known format.

In [3]:
%%time
# Transform Google location history data to a known dataframe
# Note to self: Google Location History files are big, dont load with "pd.load_json"
loch_data = json.load(open("myData.json", "r"))
loch_df = pd.DataFrame.from_dict(loch_data)
print(f"Number of Google location history points loaded: {loch_df.shape[0]:,}.")

# Extract timestamp, long, lat, accuracy and datetime from Location History.
# For future use and debugging, the timestamp is also converted to a datetime.
# Note, timestamp is in mS, long and lat must be devided by 1e7.
loch_df['timestamp'] = loch_df.locations.map(lambda x: int(x['timestampMs'])) # Not really used
loch_df['longitude'] = loch_df.locations.map(lambda x: x['longitudeE7']/1e7)
loch_df['latitude'] = loch_df.locations.map(lambda x: x['latitudeE7']/1e7)
loch_df['accuracy'] = loch_df.locations.map(lambda x: x['accuracy'])
loch_df['datetime'] = loch_df.timestamp.map(lambda x: dt.datetime.fromtimestamp(x/1000))

# Drop the rest of the dataset that is now irrelevent.
loch_df= loch_df.drop(labels=['locations'], axis=1, inplace=False)

# Display subset of data
print(f"Timeline: from {loch_df.datetime.min().strftime('%d/%m/%Y %H:%M')} to {loch_df.datetime.max().strftime('%d/%m/%Y %H:%M')}")
print(f"Accuracy: min {loch_df.accuracy.min()}, max {loch_df.accuracy.max()}")
display(loch_df)

Number of Google location history points loaded: 7.
Timeline: from 11/12/2019 15:00 to 16/03/2020 11:37
Accuracy: min 10, max 400


,timestamp,longitude,latitude,accuracy,datetime
0,1584095754000,34.863333,32.190045,52,2020-03-13 10:35:54
1,1584095754000,34.863316,32.190105,400,2020-03-13 10:35:54
2,1584079880000,34.807634,32.154875,10,2020-03-13 06:11:20
3,1584004271000,35.531036,32.787232,15,2020-03-12 09:11:11
4,1584358620000,35.022110,30.818705,16,2020-03-16 11:37:00
5,1583914880000,34.907804,32.343175,10,2020-03-11 08:21:20
6,1576076400000,34.797633,31.269703,10,2019-12-11 15:00:00


CPU times: user 33.5 ms, sys: 27.6 ms, total: 61.1 ms
Wall time: 56 ms


### MOH data normalization 
A very band and lazy attempt at normalizing the MOH dataset, there are so many edge cases.
In order to get a (maybe?) correct duration time, the date component of the timestamp is used with a strptime parsed string of the 
time component. The only edge case which is handled here is when the end datetime time component is 00:00, I manualy add a delta of 1 day to the datetime (as the MOH dont advance the date).

In [4]:
%%time
def _convertTime(time, index, start_timestamp_ms, end_timestamp_ms):
    """
    A horrible function to try and convert the disguising MOH time data to something useful.
    """
    try:
        parsed_time = dt.datetime.strptime(time.split('-')[index].strip(), "%H:%M").time()
        parsed_start_date = dt.datetime.fromtimestamp(start_timestamp_ms/1000).date()
        parsed_end_date = dt.datetime.fromtimestamp(end_timestamp_ms/1000).date()
        if index == 0:
            parsed_date = parsed_start_date
        else:
            parsed_date = parsed_end_date
        
        # Dumb MOH programmers use 00:00 to denote midnight without changing the date
        # So, if we are looking for the end date, and the time is 00:00 and the dates did not change, fix it.
        # This still does not fix the "09:00 - 02:00" problem.
        if (index == 1) and (parsed_time == dt.time(0,0)) and (parsed_start_date == parsed_end_date):
            delta = dt.timedelta(days=1)
        else:
            delta = dt.timedelta(days=0)
        
        return dt.datetime.combine(parsed_date + delta, parsed_time)
    except Exception as ex:
        # A date of 01/01/3000 marks a parse error.
        return dt.datetime(3000,1,1,0,0,0)

# Transform MOH data to a known dataframe
moh_data = json.load(open("govData.json",  "r")) 
moh_df = pd.DataFrame.from_dict(moh_data)  
print(f"Number of MOH points loaded: {moh_df.shape[0]:,}.")

# Extract id, long, lat, type, name, place, comments, fixed format times
# Not exactly sure what "type" is, assuming will allow polygons/shapes in the future.
moh_df['id'] = moh_df['features'].map(lambda x: x['id'])
moh_df['longitude'] = moh_df['features'].map(lambda x: x['geometry']['coordinates'][0])
moh_df['latitude'] = moh_df['features'].map(lambda x: x['geometry']['coordinates'][1])
moh_df['type'] = moh_df['features'].map(lambda x: x['geometry']['type'])
moh_df['iname'] = moh_df['features'].map(lambda x: x['properties']['Name'])
moh_df['place'] = moh_df['features'].map(lambda x: x['properties']['Place'])
moh_df['comments'] = moh_df['features'].map(lambda x: x['properties']['Comments'])
# Note to self: The time convertion can be done better, much much better.
moh_df['start_datetime'] = moh_df['features'].map(lambda x: _convertTime(x['properties']['stayTimes'], 0, x['properties']['fromTime'], x['properties']['toTime']))
moh_df['end_datetime'] = moh_df['features'].map(lambda x: _convertTime(x['properties']['stayTimes'], 1, x['properties']['fromTime'], x['properties']['toTime']))
# Debug datetime columns, not used for any referencing.
moh_df['debug_stayTimes'] = moh_df['features'].map(lambda x: x['properties']['stayTimes'])
moh_df['debug_fromTime'] = moh_df['features'].map(lambda x: dt.datetime.fromtimestamp(x['properties']['fromTime']/1000))
moh_df['debug_toTime'] = moh_df['features'].map(lambda x: dt.datetime.fromtimestamp(x['properties']['fromTime']/1000))

# Drop the rest of the dataset
moh_df = moh_df.drop(labels=['features'], axis=1, inplace=False)

# Dispay subset of data and count and subset of bad time information
print(f"Timeline: from {moh_df.start_datetime.min().strftime('%d/%m/%Y %H:%M')} to {moh_df.end_datetime[moh_df.start_datetime < dt.datetime(3000,1,1,0,0,0)].max().strftime('%d/%m/%Y %H:%M')}")  
bad_datetimes = moh_df[moh_df.start_datetime >= moh_df.end_datetime]
no_bad_datetimes = bad_datetimes.shape[0]
unknown_datetimes = moh_df[moh_df.start_datetime == dt.datetime(3000,1,1,0,0,0)]
no_unknown_datetimes = unknown_datetimes.shape[0]
print(f"Bad start/end datetimes: {no_bad_datetimes}") 
print(f"Unknown datetimes: {no_unknown_datetimes}") 
print(f"Total valid datapoints: {moh_df.shape[0] - no_bad_datetimes - no_unknown_datetimes}")
print("-"*40)
print("Subset of valid datapoints:")
display(moh_df)
print("Subset of bad start/end datapoints:")
display(bad_datetimes)
print("Subset of unknown datetime datapoints:")
display(unknown_datetimes)


Number of MOH points loaded: 1,301.
Timeline: from 10/03/2020 00:00 to 23/03/2020 10:30
Bad start/end datetimes: 40
Unknown datetimes: 6
Total valid datapoints: 1255
----------------------------------------
Subset of valid datapoints:


,type,id,longitude,latitude,iname,place,comments,start_datetime,end_datetime,debug_stayTimes,debug_fromTime,debug_toTime
0,Point,18853,35.295659,32.601246,,"אולמי אצולת העמק, עפולה",None,2020-03-10 00:00:00,2020-03-10 00:30:00,00:00-00:30,2020-03-10 20:00:00,2020-03-10 20:00:00
1,Point,18854,35.208909,31.752244,,מועדון האומן בירושלים,None,2020-03-10 00:00:00,2020-03-10 01:30:00,00:00-01:30,2020-03-10 20:00:00,2020-03-10 20:00:00
2,Point,18855,34.973672,32.760481,,"בית חב""ד טירת הכרמל",None,2020-03-10 00:00:00,2020-03-10 01:00:00,00:00-01:00,2020-03-10 20:00:00,2020-03-10 20:00:00
3,Point,18856,34.762707,32.036031,,בית חולים וולפסון,None,2020-03-10 00:00:00,2020-03-10 23:59:00,00:00-23:59,2020-03-10 20:00:00,2020-03-10 20:00:00
4,Point,18857,35.212663,31.772099,,מרפאת מטודלה של קופת חולים כללית ירושלים,None,2020-03-10 00:00:00,2020-03-10 23:59:00,00:00-23:59,2020-03-10 20:00:00,2020-03-10 20:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
1296,Point,20148,34.808940,31.975455,,מסעדת הסוהו - ראשון לציון,None,2020-03-11 20:00:00,2020-03-11 21:30:00,20:00 - 21:30,2020-03-11 20:00:00,2020-03-11 20:00:00
1297,Point,20149,35.213004,31.753488,,סופר אושר עד - ירושלים,None,2020-03-16 08:00:00,2020-03-16 08:00:00,08:00 - 08:00,2020-03-16 20:00:00,2020-03-16 20:00:00
1298,Point,20150,35.234619,31.837792,,"קופ""ח מאוחדת - ירושלים",None,2020-03-15 09:00:00,2020-03-15 10:00:00,09:00 - 10:00,2020-03-15 20:00:00,2020-03-15 20:00:00
1299,Point,20151,35.234619,31.837792,,"קופ""ח מאוחדת - ירושלים",None,2020-03-15 09:00:00,2020-03-15 11:00:00,09:00 - 11:00,2020-03-15 20:00:00,2020-03-15 20:00:00


Subset of bad start/end datapoints:


,type,id,longitude,latitude,iname,place,comments,start_datetime,end_datetime,debug_stayTimes,debug_fromTime,debug_toTime
6,Point,18859,34.894108,32.260483,,"מסעדת JEMS בקניון דרורים, לב השרון גלבוע",None,2020-03-10 19:00:00,2020-03-10 01:00:00,19:00-01:00,2020-03-10 20:00:00,2020-03-10 20:00:00
28,Point,18881,34.820654,32.090257,,"תחנת ז'בוטינסקי פינת קריניצי ר""ג",None,3000-01-01 00:00:00,3000-01-01 00:00:00,לא ידוע,2020-03-10 20:00:00,2020-03-10 20:00:00
29,Point,18882,34.866730,32.093136,,"תחנת הקניון הגדול פ""ת",None,3000-01-01 00:00:00,3000-01-01 00:00:00,לא ידוע,2020-03-10 20:00:00,2020-03-10 20:00:00
30,Point,18883,34.866730,32.093136,,"תחנת הקניון הגדול פ""ת",None,3000-01-01 00:00:00,3000-01-01 00:00:00,לא ידוע,2020-03-10 20:00:00,2020-03-10 20:00:00
31,Point,18884,34.821915,32.090740,,"תחנת בן גוריון פינת ז'בוטינסקי ר""ג",None,3000-01-01 00:00:00,3000-01-01 00:00:00,לא ידוע,2020-03-10 20:00:00,2020-03-10 20:00:00
99,Point,18951,34.610970,31.526631,,שטראוס שדרות - שדרות,None,2020-03-10 10:00:00,2020-03-07 13:00:00,10:00 - 13:00,2020-03-10 20:00:00,2020-03-10 20:00:00
193,Point,19045,35.295659,32.601246,,"אולמי ""אצולת העמק"", עפולה",None,2020-03-10 19:30:00,2020-03-10 00:30:00,19:30-00:30,2020-03-10 20:00:00,2020-03-10 20:00:00
226,Point,19078,35.225670,31.762253,,יס פלאנט ירושלים,None,3000-01-01 00:00:00,3000-01-01 00:00:00,לא ידוע,2020-03-11 20:00:00,2020-03-11 20:00:00
269,Point,19121,35.151196,31.656429,,בית ספר עושה חיל - אפרת,None,2020-03-11 08:00:00,2020-03-11 08:00:00,08:00 - 08:00,2020-03-11 20:00:00,2020-03-11 20:00:00
291,Point,19143,34.884095,31.432154,,גנים טרום חובה וחובה - שומריה,None,2020-03-11 10:00:00,2020-03-11 10:00:00,10:00 - 10:00,2020-03-11 20:00:00,2020-03-11 20:00:00


Subset of unknown datetime datapoints:


,type,id,longitude,latitude,iname,place,comments,start_datetime,end_datetime,debug_stayTimes,debug_fromTime,debug_toTime
28,Point,18881,34.820654,32.090257,,"תחנת ז'בוטינסקי פינת קריניצי ר""ג",None,3000-01-01 00:00:00,3000-01-01 00:00:00,לא ידוע,2020-03-10 20:00:00,2020-03-10 20:00:00
29,Point,18882,34.866730,32.093136,,"תחנת הקניון הגדול פ""ת",None,3000-01-01 00:00:00,3000-01-01 00:00:00,לא ידוע,2020-03-10 20:00:00,2020-03-10 20:00:00
30,Point,18883,34.866730,32.093136,,"תחנת הקניון הגדול פ""ת",None,3000-01-01 00:00:00,3000-01-01 00:00:00,לא ידוע,2020-03-10 20:00:00,2020-03-10 20:00:00
31,Point,18884,34.821915,32.090740,,"תחנת בן גוריון פינת ז'בוטינסקי ר""ג",None,3000-01-01 00:00:00,3000-01-01 00:00:00,לא ידוע,2020-03-10 20:00:00,2020-03-10 20:00:00
226,Point,19078,35.225670,31.762253,,יס פלאנט ירושלים,None,3000-01-01 00:00:00,3000-01-01 00:00:00,לא ידוע,2020-03-11 20:00:00,2020-03-11 20:00:00
579,Point,19430,35.163635,31.679012,,"רחוב הדבש 13, אפרת",None,3000-01-01 00:00:00,3000-01-01 00:00:00,לא ידוע,2020-03-13 20:00:00,2020-03-13 20:00:00


CPU times: user 89 ms, sys: 31.2 ms, total: 120 ms
Wall time: 110 ms


In [5]:
# Make a slice of only the relevant  dates out of the location history DataFrame.
loch_slice = loch_df[(loch_df.datetime >=  moh_df.start_datetime.min()) & (loch_df.datetime <= moh_df.end_datetime[moh_df.end_datetime < dt.datetime(3000,1,1,0,0,0)].max())]
print(f"Cross referencing with location history from {loch_slice.datetime.min().strftime('%d/%m/%Y %H:%M')} to {loch_slice.datetime.max().strftime('%d/%m/%Y %H:%M')}")

Cross referencing with location history from 11/03/2020 08:21 to 16/03/2020 11:37


## The super duper deoptimized cross referencing loops
I cant really explain this, but this is what I think I did:
1. Create master results DataFrame.
2. For each row of the MOH exposure points:
    - Do some time sanity checks, if time is wierd, put it aside.
    - Get the time relevant slice of location history.
    - For each location history point, mesure the distance and add to a private dataframe
    - Take the results with minimum distance and the minimum accuracy and combine them into a sngle result row in the master results DataFrame.
3. Display subset of master results DataFrame.

> Note: There is no reason to create the internal DataFrame, I did it this way so later I could use the DataFrames as sample data to play with pandas.

In [6]:
%%time
TIME_BUFFER = dt.timedelta(minutes=0)
# Counters for bad datapoints
# Note: These were counters, now they are lists.
counters = { 'unknown_datetime' : [], 'bad_datetime' : [], 'missing_results' : [], 'debug_timestamp' : [], 'totaly_failed' : []}
# Total dataset size
frame_size = moh_df.shape[0]

# Do I really need to create this DataFrame? No.
results_df = pd.DataFrame(columns=['incident_id', 'incident_time', 'incident_location', 'incident_name','incident_place', 'incident_comments', 'min_distance_location','min_distance_accuracy', 'min_distance_distance', 'min_accuracy_accuracy'])
for inc_idx, inc_row in moh_df.iterrows():
    # Everything is in a try, whay? because MOD data can lie!
    try:
        # Time buffers! may be useful in the future
        search_start_datetime = inc_row.start_datetime - TIME_BUFFER
        search_end_datetime = inc_row.end_datetime + TIME_BUFFER
        
        # Some sanity checks
        if inc_row.start_datetime > inc_row.end_datetime:
            counters['bad_datetime'].append(inc_row)
            continue
        if inc_row.start_datetime == dt.datetime(3000,1,1,0,0,0):
            counters['unknown_datetime'].append(inc_row)
            continue
            # Select the whole timerange
            search_start_datetime = datetime.combine(search_start_datetime.date(), dt.time(0,0))
            search_end_datetime = datetime.combine(search_end_datetime.date() + dt.timedelta(days=1), dt.time(0,0))
        # Debug, just to see how many timestams are *maybe* correct
        if inc_row.debug_fromTime != inc_row.debug_toTime:
            counters['debug_timestamp'].append(inc_row)

        # Get all locations from location history that corrspond to the datetime
        loch_results = loch_slice[(loch_slice.datetime >= search_start_datetime ) & (loch_slice.datetime <= search_end_datetime)]
        if loch_results.empty:
            # No results, manual verification :(
            counters['missing_results'].append(inc_row)
            continue

        # We can only set a distance and index, but it's easy to run slicings on a clean DataFrame
        mesured_distances = pd.DataFrame(columns=['index','datetime', 'distance', 'latitude', 'longitude', 'accuracy'])
        for loc_idx, loc_row in loch_results.iterrows():
            distance = geopy.distance.geodesic((inc_row.longitude, inc_row.latitude),(loc_row.longitude, loc_row.latitude))
            mesured_distances = mesured_distances.append({ 'index' : loc_idx,
                                                           'datetime' : loc_row.datetime,
                                                           'distance' : distance.km,
                                                           'latitude' : loc_row.longitude,
                                                           'longitude' : loc_row.latitude,
                                                           'accuracy' : loc_row.accuracy}, ignore_index=True)
        
        min_distance_df = mesured_distances.sort_values(by=['distance'], ).head(1)
        min_accuracy_df = mesured_distances.sort_values(by=['accuracy'], ).head(1)
        
        results_df = results_df.append({'incident_id' : inc_row.id, 
                                  'incident_time' : str(f'{inc_row.start_datetime.strftime("%d/%m/%Y %H:%M")} - {inc_row.end_datetime.strftime("%d/%m/%Y %H:%M")}'), 
                                  'incident_location' : (inc_row.latitude, inc_row.longitude), 
                                  'incident_name' : inc_row.iname,
                                  'incident_place' : inc_row.place, 
                                  'incident_comments' : inc_row.comments,
                                   # I dont know why (yet) but I needed to use here .item()
                                  'min_distance_location' : (min_distance_df.latitude.item(), min_distance_df.longitude.item()), 
                                  'min_distance_accuracy' : min_distance_df.accuracy.item(),
                                  'min_distance_distance' : min_distance_df.distance.item(),
                                  'min_accuracy_location' : (min_accuracy_df.latitude.item(), min_accuracy_df.longitude.item()), 
                                  'min_accuracy_distance' : min_accuracy_df.distance.item(),
                                  'min_accuracy_accuracy' : min_accuracy_df.accuracy.item()}, ignore_index=True)
    except Exception as ex:
        # Yep, the data lied.
        print(f"You have 1 new exception: {ex}")
        counters['totaly_failed'].append(inc_row)
    
    # A poor mans progress meter.
    if inc_idx % 100 == 0:
        print(f"\rProcessed {inc_idx}/{frame_size} exposure incidents (estimate)", end='')
print(" Done")

print(f"Total incidents checked: {results_df.shape[0]}")
print(f"Incidents with unknown time skipped: {len(counters['unknown_datetime'])}")
print(f"Incidents with bad time skipped: {len(counters['bad_datetime'])}")
print(f"Incidents with missing results: {len(counters['missing_results'])}")
print(f"Incidents witch just failed for some reason: {len(counters['debug_timestamp'])}")
print(f"Debug timestamp incidents: {len(counters['totaly_failed'])}")
# Show some results    
display(results_df.sort_values(by=['min_distance_distance']))



Processed 1200/1301 exposure incidents (estimate) Done
Total incidents checked: 132
Incidents with unknown time skipped: 6
Incidents with bad time skipped: 18
Incidents with missing results: 1145
Incidents witch just failed for some reason: 0
Debug timestamp incidents: 0


,incident_id,incident_time,incident_location,incident_name,incident_place,incident_comments,min_distance_location,min_distance_accuracy,min_distance_distance,min_accuracy_accuracy,min_accuracy_distance,min_accuracy_location
1,10639,13/03/2020 08:00 - 13/03/2020 13:30,"(32.1912984000001, 34.863387)",,קפה שלונסקי - רעננה,,"(34.863316, 32.190105)",400,0.109408,52,0.114767,"(34.863333, 32.190045)"
87,10246,11/03/2020 08:00 - 11/03/2020 09:30,"(32.342406, 34.912319)",,מכללת רופין,בניין 30 חדר מעבדת מחשבים בקומה מינוס אחת ובספ...,"(34.907804, 32.343175)",10,0.505795,10,0.505795,"(34.907804, 32.343175)"
32,19171,11/03/2020 14:00 - 21/03/2020 17:30,"(32.0991384720001, 34.8383772510001)",,"ת""ת כרם גפן ישראל - בני ברק",None,"(34.807634, 32.154875)",10,6.134492,10,6.134492,"(34.807634, 32.154875)"
90,19838,11/03/2020 04:45 - 18/03/2020 07:00,"(32.0873548000001, 34.8289585000001)",,בית הכנסת הגדול בבני ברק מורשה ודעת - בני ברק,None,"(34.807634, 32.154875)",10,6.614877,10,6.614877,"(34.807634, 32.154875)"
110,20047,11/03/2020 06:00 - 18/03/2020 07:00,"(32.0879286270001, 34.832599612)",,בית כנסת צורטקוב - בני ברק,None,"(34.807634, 32.154875)",10,6.721861,10,6.721861,"(34.807634, 32.154875)"
...,...,...,...,...,...,...,...,...,...,...,...,...
7,19074,11/03/2020 08:00 - 11/03/2020 13:00,"(30.9878617000001, 34.9292548)",,"קופת חולים כללית, צבי בורנשטיין 1, ירוחם",None,"(34.907804, 32.343175)",10,123.868490,10,123.868490,"(34.907804, 32.343175)"
35,19271,12/03/2020 08:00 - 12/03/2020 12:00,"(31.249581, 35.0828680000001)",,"קופת חולים כללית, כסיפה א'",None,"(35.531036, 32.787232)",15,148.417630,15,148.417630,"(35.531036, 32.787232)"
99,19957,16/03/2020 09:00 - 16/03/2020 14:00,"(32.4576836000001, 35.0574311)",,"משרד טכנולוגיות מג,ב חריש - חריש",None,"(35.02211, 30.818705)",16,149.596572,16,149.596572,"(35.02211, 30.818705)"
83,19681,16/03/2020 08:00 - 16/03/2020 15:00,"(32.6037908000001, 35.2940938)",,בנק יהב עפולה,None,"(35.02211, 30.818705)",16,165.416088,16,165.416088,"(35.02211, 30.818705)"


CPU times: user 2.86 s, sys: 563 ms, total: 3.43 s
Wall time: 3.31 s


## Display a map
Display a map swith all MOH locations on it.
    - Red icons - Maybe exposed.
    - Orange icons - No cross-referencing done.
    - Green - Probably not exposed (again, hidden additional time data in the comments field).

Due to not grouping of the markers (as I want to see them all with their colors) the map is **very** sluggish. I did not intent on adding a map, but it's Jupyter and I can. 


In [7]:
%%time
CRITICAL_DISTANCE_KM = 1.0
# Create the map
m = Map(zoom=7, center=[31.771959, 35.217018],scroll_wheel_zoom=True,basemap = basemaps.OpenStreetMap.Mapnik)
# Create the icons
red_icon = AwesomeIcon(marker_color='red', name='exclamation-triangle')
green_icon = AwesomeIcon(marker_color='green', name='check')
orange_icon = AwesomeIcon(marker_color='orange', name='flag')

# Add known locations markers
known_count = len([m.add_layer(Marker(z_index_offset=50 if row.min_distance_distance < CRITICAL_DISTANCE_KM else 0,icon=red_icon if row.min_distance_distance < CRITICAL_DISTANCE_KM else green_icon,draggable=False, location=row.incident_location, title=f'מיקום: {row.incident_place}\nהערות: {row.incident_comments}\nזמן: {row.incident_time}\nמרחק: {row.min_distance_distance:0.4f} ק"מ\nדיוק: {row.min_distance_accuracy}')) for idx, row in results_df.iterrows()])
# Add unknown/bad/missing markers
unknown_count = len([m.add_layer(Marker(z_index_offset=25, icon=orange_icon, draggable=False, location=(row.latitude, row.longitude), title=f'מיקום: {row.place}\nהערות: {row.comments}\nזמן: {row.start_datetime.strftime("%d/%m/%Y %H:%M")} - {row.end_datetime.strftime("%d/%m/%Y %H:%M")}')) for row in itertools.chain.from_iterable(counters.values())])
# Alow full screen
m.add_control(FullScreenControl())
# Display some info 
print(f"Added {known_count} known markers.")
print(f"Added {unknown_count} unknown markers.")
print(f"Total markers on map: {known_count + unknown_count}/{frame_size}")
display(m)


Added 132 known markers.
Added 1169 unknown markers.
Total markers on map: 1301/1301


Map(center=[31.771959, 35.217018], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

CPU times: user 6.53 s, sys: 2.17 s, total: 8.7 s
Wall time: 8.23 s


## Save the data and the map
Saves the date and map, incase we want it later. 

In [8]:
results_df.sort_values(by=['min_distance_distance']).to_html(dt.datetime.now().strftime("data/data%Y%m%d_%H%M.html"))
embed_minimal_html(dt.datetime.now().strftime("maps/map%Y%m%d_%H%M.html"), views=[m], title='Exposure Map')

The end.